# Data Preparation

In [1]:
import pandas as pd
from text_preprocessing_funcs import *

In [2]:
df = pd.read_json('raw_data.json')
df.head()

,url,name,type,price,prefecture,smv,acidity,kw_word,kw_wines,kw_beer,kw_foods,description
0,https://www.truesake.com/collections/all/produ...,"Dassai 45 Junmai Daiginjo ""Otter Festival""",Junmai Daiginjo,30.00,Yamaguchi,+3,1.4,Chunky,Pinot Noir / Chewy Whites,Ales,"Mushroom risotto, tempura, fried chicken, cavi...",This sake has an older brother Daiginjo that h...
1,https://www.truesake.com/collections/all/produ...,"Kikusui Junmai Ginjo ""Chrysanthemum Water""",Junmai Ginjo,33.00,Niigata,+1,1.6,Citrus,Sharp Whites/Dry Reds,Ales/Ambers,Suitable for light and stronger flavored dishe...,True Sake owner Beau Timken recorded K...
2,https://www.truesake.com/collections/all/produ...,"Wakatake Onikoroshi Junmai Daiginjo ""Demon Sla...",Junmai Daiginjo,49.00,Shizuoka,+/-0,1.4,Popular,Elegant reds/Complex whites,Crisp Ales,Cuisines that like big white wines.,Talk about a nice aroma package on thi...
3,https://www.truesake.com/collections/all/produ...,"Shunnoten Tokubetsu Junmai ""Fishermen Cup""",Tokubetsu Junmai,9.00,Yamanashi,+3,1.6,Smooth,Pinot Noir/Slippery Whites,Gentle Ales,None,The nose on this awesome looking “catch cup” i...
4,https://www.truesake.com/collections/all/produ...,"Kenbishi Kuromatsu Honjozo ""Black Pine""",Honjozo,40.00,Hyogo,±0.5,1.5,Rich,Deep Reds/Fat Whites,Stouts,Perfect for pub food salty and savory and grea...,The nose on this extremely famous Honjozo from...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322 entries, 0 to 321
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          322 non-null    object 
 1   name         322 non-null    object 
 2   type         303 non-null    object 
 3   price        322 non-null    object 
 4   prefecture   318 non-null    object 
 5   smv          304 non-null    object 
 6   acidity      277 non-null    float64
 7   kw_word      288 non-null    object 
 8   kw_wines     286 non-null    object 
 9   kw_beer      285 non-null    object 
 10  kw_foods     280 non-null    object 
 11  description  307 non-null    object 
dtypes: float64(1), object(11)
memory usage: 32.7+ KB


### Sake Types

In [4]:
lam_func = lambda t: clean_text(t) if t is not None else None

df['type'] = df['type'].apply(lam_func)

In [5]:
df.type.unique()

array(['junmai daiginjo', 'junmai ginjo', 'tokubetsu junmai', 'honjozo',
       'junmai', 'honjozo nama genshu', 'ginjo', 'futsushu',
       'nigori junmai', 'junmai taru', 'ginjo nama genshu', None,
       'tokubetsu junmai nigori', 'yamahai junmai ginjo genshu',
       'yamahai junmai', 'nigori', 'daiginjo nama',
       'tokubetsu junmai genshu', 'nigori genshu', 'futsushu nama genshu',
       'hope', 'yuzu sake', 'junmai ginjo nigori', 'junmai nama', '',
       'umeshu', 'nigori junmai daiginjo', 'yamahai junmai daiginjo',
       'junmai yuzu', 'daiginjo', 'muroka junmai daiginjo nama genshu',
       'sparkling', 'tokubesu junmai', 'ginjo sparkling nigori',
       'junmai nigori', 'kimoto junmai daiginjo', 'cooking sake',
       'junmai kimoto', 'sparkling junmai ginjo',
       'junmai daiginjo kimoto', 'muroka junmai daiginjo',
       'nigori junmai ginjo nama genshu', 'junmai ginjo nama',
       'junmai ginjo muroka nama genshu', 'futsushu taru', 'kijoshu',
       'junmai daiginjo

### Prefectures

In [6]:
df['prefecture'] = df['prefecture'].apply(lam_func)

In [7]:
df.prefecture.unique()

array(['yamaguchi', 'niigata', 'shizuoka', 'yamanashi', 'hyogo', 'osaka',
       'kouchi', 'hokkaido', 'yamagata', 'tottori', 'tochigi', 'fukui',
       'akita', 'tokushima', 'ohita', 'wakayama', 'shimane', 'ishikawa',
       'fukushima', 'aomori', 'kyoto', 'iwate', 'saitama', 'gifu',
       'nagano', 'oakland california', 'shiga', 'ehime', 'kochi',
       'miyagi', 'ibaraki', 'aichi', 'hiroshima', 'saga', 'nanago',
       'fukuoka', 'nara', 'domestic san francisco', None, 'okayama',
       'okinawa', 'tokyo', 'shizouka', 'gunma', 'kanagawa'], dtype=object)

I'll consolidate Oakland and San Francisco into just California. 

In [8]:
cali_dict = {'oakland california': 'california',
            'domestic san francisco': 'california'}

df['prefecture'] = df['prefecture'].replace(cali_dict, regex=True)

### Keywords and Description Columns

In [9]:
cols_to_fix_dict = {
    'kw_word': ['WINE'],
    'kw_wines': ['BEER'],
    'kw_beer': ['FOOD', 'BEER:', 'Foods:'],
    'kw_foods': ['Updated', '('],
    'description': ['WORD']
}

In [10]:
def clean_df_text_cols(df, cols_to_fix_dict):
    for col, text_list in cols_to_fix_dict.items():
        if col in df.columns.tolist():
            for text in text_list:   
                # Apply split_on_chars func to parse for the right text
                df[col] = df[col].apply(lambda d: split_on_chars(text, d))
            # Clean text
            df[col] = df[col].apply(lambda d: clean_text(d))   
    return df

In [11]:
df = clean_df_text_cols(df, cols_to_fix_dict)

### Price

In [12]:
df.price.unique()

array(['30.00', '33.00', '49.00', '9.00', '40.00', '8.00', '10.00',
       '23.00', '5.00', '38.00', '43.00', '90.00', '13.00', '24.00',
       '17.00', '18.00', '46.00', '3.00', '36.00', '32.00', '34.00',
       '6.00', '58.00', '28.00', '37.00', '27.00', '11.00', '21.00',
       '89.00', '29.00', '44.00', '7.00', '39.00', '22.00', '12.00',
       '35.00', '4.00', '16.00', '25.00', '117.00', '48.00', '57.00',
       '55.00', '80.00', '45.00', '71.00', '14.00', '26.00', '60.00',
       '54.00', '81.00', '127.00', '31.00', '68.00', '87.00', '100.00',
       'urrent price        $25.00', '50.00', '76.00', '66.00', '79.00',
       '102.00', '110.00', '142.00', '20.00', '109.00', '41.00', '145.00',
       '161.00', '56.00', '129.00', '42.00', '53.00', '51.00', '82.00',
       '88.00', '59.00', '135.00', 'urrent price        $32.00', '70.00',
       '182.00', '19.00', '47.00', '267.00', '150.00', '74.00', '133.00',
       '215.00', 'urrent price        $36.00', '700.00', '52.00', '92.00',
 

In [13]:
df['price'] = df['price'].apply(lambda x: float(x[-4:]) if '$' in x else float(x))

### SMV (Sake Meter Value)

In [14]:
df.smv.unique()

array(['+3', '+1', '+/-0', '±0.5', '+4', '+7', '+10', '-2', '+5', '+2.0',
       '+6', '-11', '-1.5', None, '+3.5', '-100', '+2', '-4', '+12', '+8',
       '-22', 'N/A', '-70', '-35', '-3', '+3.6', '-50', '-3.0', '1.3',
       '+2.5', '+/-0 ~ +2', '-20', '-82', '-12', '-70 ~ -90', '-25', '-1',
       '+1.5', '±0', '-0.6', '-70~-80', '-44', '-6/-7', '-0.5 ~+0.5',
       '-10', '+1~ +2', '-96', '±1', '-60', '-80', '-23', '+0.5', '-65',
       '-15', '+15.5', '-40', 'NA', '+4.0', '-14', '-13', '3.5', '-8.5',
       '+1.4', '-33', '-5', '+1.2', '-4.0', '-21', '-49'], dtype=object)

This function should take care of most of the conversions and then I'll see what's left:

In [15]:
def clean_smvs(val):
    try:
        if '±' in val:
            val = re.sub('±', '', val)
        val = float(val)
    except Exception as e:
        val = val
    return val

In [16]:
[clean_smvs(val) for val in df.smv.unique() if isinstance(clean_smvs(val), float) is False]

['+/-0',
 None,
 'N/A',
 '+/-0 ~ +2',
 '-70 ~ -90',
 '-70~-80',
 '-6/-7',
 '-0.5 ~+0.5',
 '+1~ +2',
 'NA']

Only a few -- can simply do this with a dictionary and the `.replace` method.

In [17]:
smv_dict = {
    '\+/-0': 0.0,
    '\+/-0 ~ \+2': 1.0,
    '-70 ~ -90': -80.0,
    '-70~-80': -75.0,
    '-6/-7': -6.5,
    '-0.5 ~\+0.5': 0.0,
    '\+1~ \+2': 1.5,
    'NA': None,
    'N/A': None
}

In [18]:
df['smv'] = df['smv'].map(clean_smvs)
df['smv'] = df['smv'].replace(smv_dict, regex=True)

## Fill NA's

In [19]:
# Fill NA Values
df.type.fillna('unknown', inplace=True)
df.prefecture.fillna('unknown', inplace=True)
df.smv.fillna(df.smv.mean(), inplace=True)
df.acidity.fillna(df.acidity.mean(), inplace=True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322 entries, 0 to 321
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          322 non-null    object 
 1   name         322 non-null    object 
 2   type         322 non-null    object 
 3   price        322 non-null    float64
 4   prefecture   322 non-null    object 
 5   smv          322 non-null    float64
 6   acidity      322 non-null    float64
 7   kw_word      322 non-null    object 
 8   kw_wines     322 non-null    object 
 9   kw_beer      322 non-null    object 
 10  kw_foods     322 non-null    object 
 11  description  322 non-null    object 
dtypes: float64(3), object(9)
memory usage: 32.7+ KB


## Save for later

In [21]:
# Save dataframe as json file to be used later!
# df.to_json('prepared_data.json')